In [3]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set up constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50

# Define the path to the images folder
data_dir = '/kaggle/input/images-skinlesion/images'
class_names = ['MEL', 'NV', 'BCC', 'SCC']

# Set up data generators with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=class_names,
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=class_names,
    shuffle=False
)

# Load pre-trained InceptionResNetV2 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))


Found 7174 images belonging to 4 classes.
Found 1791 images belonging to 4 classes.


In [4]:

# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, kernel_regularizer='l2', activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weight_dict = dict(enumerate(class_weights))

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('/kaggle/working/best_model_inceptionresnetv2.keras', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [5]:

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weight_dict,
    callbacks=[reduce_lr, early_stopping, model_checkpoint]
)

model.save('/kaggle/working/skin_lesion_classifier_inceptionresnetv2_final.keras')

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727620451.946578     113 service.cc:145] XLA service 0x7b5f78002790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727620451.946641     113 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1727620451.946647     113 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1727620541.916698     113 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_105', 28 bytes spill stores, 28 bytes spill loads
ptxas warning : Registers 

220/224 ━━━━━━━━━━━━━━━━━━━━ 2s 695ms/step - accuracy: 0.6101 - loss: 3.1957

I0000 00:00:1727620765.703689     111 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion', 72 bytes spill stores, 72 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_566', 16 bytes spill stores, 16 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_117', 40 bytes spill stores, 40 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_118', 28 bytes spill stores, 28 bytes spill loads



224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6113 - loss: 3.1896
Epoch 1: val_accuracy improved from -inf to 0.81136, saving model to /kaggle/working/best_model_inceptionresnetv2.keras
224/224 ━━━━━━━━━━━━━━━━━━━━ 460s 1s/step - accuracy: 0.6116 - loss: 3.1881 - val_accuracy: 0.8114 - val_loss: 2.5349 - learning_rate: 1.0000e-04
Epoch 2/50
  1/224 ━━━━━━━━━━━━━━━━━━━━ 2:19 624ms/step - accuracy: 0.7500 - loss: 2.2998

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_accuracy did not improve from 0.81136
224/224 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.7500 - loss: 2.2998 - val_accuracy: 0.1935 - val_loss: 5.6320 - learning_rate: 1.0000e-04
Epoch 3/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 550ms/step - accuracy: 0.7856 - loss: 2.2282
Epoch 3: val_accuracy improved from 0.81136 to 0.83182, saving model to /kaggle/working/best_model_inceptionresnetv2.keras
224/224 ━━━━━━━━━━━━━━━━━━━━ 217s 704ms/step - accuracy: 0.7856 - loss: 2.2278 - val_accuracy: 0.8318 - val_loss: 1.9596 - learning_rate: 1.0000e-04
Epoch 4/50
  1/224 ━━━━━━━━━━━━━━━━━━━━ 2:12 593ms/step - accuracy: 0.7812 - loss: 2.3530
Epoch 4: val_accuracy did not improve from 0.83182
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 800us/step - accuracy: 0.7812 - loss: 2.3530 - val_accuracy: 0.3226 - val_loss: 4.3674 - learning_rate: 1.0000e-04
Epoch 5/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - accuracy: 0.8015 - loss: 1.7873
Epoch 5: val_accuracy improved from 0.83182 to 0.84148, saving mo

In [6]:

# Evaluate the model
test_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Get predictions
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('/kaggle/working/confusion_matrix.png')
plt.close()

# Print classification report
print(classification_report(y_true, y_pred, target_names=class_names))

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('/kaggle/working/training_history.png')
plt.close()

# Save the final model

print("Training completed. Model saved, confusion matrix and training history plots generated.")

Found 8965 images belonging to 4 classes.
281/281 ━━━━━━━━━━━━━━━━━━━━ 168s 564ms/step
              precision    recall  f1-score   support

         MEL       0.00      0.00      0.00       514
          NV       0.01      0.04      0.01      1113
         BCC       0.04      0.00      0.01      6705
         SCC       0.97      0.89      0.93       633

    accuracy                           0.07      8965
   macro avg       0.25      0.23      0.24      8965
weighted avg       0.10      0.07      0.07      8965

Training completed. Model saved, confusion matrix and training history plots generated.
